In [0]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

In [0]:
g_drive = 'drive/My Drive/Colab Notebooks/Piman2/'

In [0]:
# グーグルドライブから必要なファイルを取得
# 画像も取得しているのはv3の時にエラーが出ため、このコードだと必要ないのかも
import shutil
import glob
import os
shutil.copyfile(g_drive + "model_data/utils.py", "./utils.py")
shutil.copyfile(g_drive + "model_data/keras_yolo.py", "./keras_yolo.py")
shutil.copyfile(g_drive + "model_data/keras_darknet19.py", "./keras_darknet19.py")
shutil.copyfile(g_drive + "model_data/yolo_utils.py", "./yolo_utils.py")
shutil.copyfile(g_drive + "model_data/FiraMono-Medium.otf", "./FiraMono-Medium.otf")

files = glob.glob(g_drive + "images/*")
os.mkdir("./images")
for file in files:
     shutil.copyfile(file, "./images/" + os.path.basename(file))

In [0]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from PIL import Image, ImageDraw, ImageFont
import time
import cv2

import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from keras_yolo import yolo_head, yolo_eval, preprocess_true_boxes, yolo_loss, yolo_body,tiny_yolo_body

from keras.layers import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.regularizers import l2
from keras.optimizers import SGD, Adam

In [0]:
tf.test.gpu_device_name()

In [0]:
#YOLO モデルの設定
image_size = 384
dataname = 'Piman'
class_names = [dataname]
YOLO_ANCHORS = np.array(
    ((0.57273, 0.677385), (1.87446, 2.06253), (3.33843, 5.47434),
     (7.88282, 3.52778), (9.77052, 9.16828)))
anchors = YOLO_ANCHORS

## load images

In [0]:
xml_df = pd.read_csv(g_drive + '/labels.csv')
fileName = np.sort(xml_df["filename"].unique())

In [0]:
images_dir = g_drive + '/images/'
images_f = open(g_drive + '/labels.csv','r')
images = []
images_cv = []
count = 0

for fn in fileName:
      img = cv2.imread(images_dir+fn)
      orig_size = np.array([img.shape[1], img.shape[0]])
      img = cv2.resize(img, (image_size,image_size), interpolation=cv2.INTER_CUBIC)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      image_data = np.array(img, dtype=np.float)
      image_data /= 255.
      images.append(image_data)
      images_cv.append(img)
    
images_f.close()
orig_size = np.expand_dims(orig_size, axis=0)
print (orig_size)
images = np.asarray(images)
print(images.shape)

## load labels

In [0]:
# Load Labels
labels_path = g_drive + '/labels.csv'
labels_f = open(labels_path,'r')
boxes = []
detectors_mask_list = []
matching_true_boxes_list = []


textAry = []
for fn in fileName:
  count = 0
  textLine = 'images/' + fn
  tempDf = xml_df[xml_df['filename'] == fn]
  for index, d in tempDf.iterrows():
    textLine =  ("%s %d,%d,%d,%d,0" % (textLine, d.xmin, d.xmax, d.ymin, d.ymax))
    #print(textLine)
    box_xy = [[(float(d.xmax) + float(d.xmin))*0.5,(float(d.ymax) + float(d.ymin))*0.5]]
    box_wh = [[(float(d.xmax) - float(d.xmin)),(float(d.ymax) - float(d.ymin))]]
    box_xy = box_xy / ((orig_size)*1.0)
    box_wh = box_wh / ((orig_size)*1.0)
    if count==0:
      box = np.concatenate((box_xy, box_wh, np.array([[1]])), axis=1)
    else:
      box = np.append(box, np.concatenate((box_xy, box_wh, np.array([[1]])), axis=1), axis=0)
    count = count + 1
  if len(box) < 10:
      box = np.append(box, np.zeros((10-len(box), 5)), axis=0)
  boxes.append(box)
  #print(box.shape)
  detectors_mask, matching_true_boxes = preprocess_true_boxes(box, anchors, [image_size, image_size])
  detectors_mask_list.append(detectors_mask)
  matching_true_boxes_list.append(matching_true_boxes)
    
# Precompute detectors_mask and matching_true_boxes for training.
# Detectors mask is 1 for each spatial position in the final conv layer and
# anchor that should be active for the given boxes and 0 otherwise.
# Matching true boxes gives the regression targets for the ground truth box
# that caused a detector to be active or 0 otherwise.
detectors_mask_shape = np.asarray(detectors_mask_list[0]).shape
matching_boxes_shape = np.asarray(matching_true_boxes_list[0]).shape

## Define Model

In [0]:
# Create model input layers.
image_input = Input(shape=(image_size, image_size, 3))
boxes_input = Input(shape=(None, 5))
detectors_mask_input = Input(shape=detectors_mask_shape)
matching_boxes_input = Input(shape=matching_boxes_shape)

boxes = np.asarray(boxes)
detectors_mask_list = np.asarray(detectors_mask_list)
matching_true_boxes_list = np.asarray(matching_true_boxes_list)


model_body = tiny_yolo_body(image_input, len(anchors), len(class_names))

In [0]:
with tf.device('/cpu:0'):
    # TODO: Replace Lambda with custom Keras layer for loss.
    model_loss = Lambda(
        yolo_loss,
        output_shape=(1, ),
        name='yolo_loss',
        arguments={'anchors': anchors,
                   'num_classes': len(class_names),
                   'print_loss': True})([
                       model_body.output, 
                       boxes_input,
                       detectors_mask_input, 
                       matching_boxes_input
                   ])
model = Model(
    [image_input, boxes_input, detectors_mask_input,
     matching_boxes_input], model_loss)
sgd = SGD(lr=0.0000005, decay=0.0005, momentum=0.9)
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam, 
              loss={
                  'yolo_loss': lambda y_true, y_pred: y_pred
              })  # This is a hack to use the custom loss function in the last layer.
#model.summary()

In [0]:

num_steps = 100
# TODO: For full training, put preprocessing inside training loop.
#for i in range(num_steps):
#     loss = model.train_on_batch(
#         [image_data, boxes, detectors_mask, matching_true_boxes],
#         np.zeros(len(image_data)))

#model.load_weights(g_drive + 'tiny_weights.h5')
model.fit([images, boxes, detectors_mask_list, matching_true_boxes_list],
          np.zeros(len(images)),
          batch_size=1,
          epochs=num_steps)
model.save_weights(g_drive + 'tiny_weights.h5')